In [ ]:
import deepxde as dde
import numpy as np
import yaml
import pandas as pd
import time
import datetime
import os
import torch
from scipy.special import gamma, lambertw
import matplotlib.pyplot as plt
from PIL import Image
from utils.Mycallbacks import *

# Set default tensor type to cuda tensor
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Set copy flag to 0 and directory path to the YZ directory
CopyFlag = 0
YZ_dir_path = './result/Forward/SinglePump/'

# Load config file
with open('./Config/DoubleBeam.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

# Load sample number and YZ directory path from the config file
sample_num = config['sample_num']
YZ_dir_path = config['YZ_dir_path']

# Initialize constants
N = 1000 # bounces number
T = 180
D = 0.7098 #cm²/s
I = 5*10 # kg*(cm)^2/s^3
q = 5
Rwall = 3.1135  #1/s
Rsd = 15.017   #1/s
Rrel = Rsd + q*Rwall
k1 = 10**(-20)*10**8 # kg*cm^4/s^2
k2 = 10**(-17)*10**4 # cm^2
k3 = 1 # cm^-3
sigm = 2.34601*10**(-13) # cm^2
n = 6.0952E13 #1/cm³
rcell = 1.5
rpump = 0.75
vKK = 701.27 * 100 #cm/s

def bloch_pde(x_in, y_in):
    ''' x_in: (x,y,z) axis
        y_in: Pz polarization
    '''
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]

    dy_xx = dde.grad.hessian(y_in, x_in, i=0, j=0)
    dy_yy = dde.grad.hessian(y_in, x_in, i=1, j=1)
    dy_zz = dde.grad.hessian(y_in, x_in, i=2, j=2) # dy/dxidxj

    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2)) #双向泵浦光
    Rop0_z = np.asarray(Rop0_z)    

    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    Rop = torch.from_numpy(Rop).cuda()

    return -D*(dy_xx+dy_yy+dy_zz) + ((Rop+Rsd)/q)*y_in - Rop/q

def func_boundary(x_in, Pz, numpy_X):
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]
    
    dPz_x = dde.grad.jacobian(Pz, x_in, i=0, j=0) # This is right
    dPz_y = dde.grad.jacobian(Pz, x_in, i=0, j=1)
    dPz_z = dde.grad.jacobian(Pz, x_in, i=0, j=2)
    
    #Rop = Rop0 * np.exp(-2*(x**2+y**2)/rpump**2)
    
    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2)) #双向泵浦光
    Rop0_z = np.asarray(Rop0_z)      
    
    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    
    #second_term = np.sqrt(1/2*np.abs(Rop*D))
    second_term = vKK/(2*N*(2-1/N))
    
    # tranfer back to torch.tensor
    #second_term = torch.from_numpy(second_term).cuda()
    Rop = torch.from_numpy(Rop).cuda()
    x = torch.from_numpy(x).cuda()
    y = torch.from_numpy(y).cuda()
    z = torch.from_numpy(z).cuda()
    
    return (D/rcell*(x*dPz_x + y*dPz_y + z*dPz_z) + Pz*second_term)




In [ ]:
# Define the geometry of the sphere
geom = dde.geometry.Sphere([0, 0, 0], rcell)

# Define the boundary condition
bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary)

# Define the data for the PDE
data = dde.data.PDE(geom, bloch_pde, bc, num_domain=20000, num_boundary=2000)

# Create the directory to save the model in
ModelSaveDir = config['ModelSaveDir']
if not os.path.exists(ModelSaveDir):
    os.makedirs(ModelSaveDir)

# Define the network depth and width
depth = 4
width = 120

# Define the neural network
net = dde.nn.FNN([3] + [width] * depth + [1], "tanh", "Glorot normal")

# Define the callbacks for saving the model and early stopping
myCallback1 = saveModel(ModelSaveDir,'DoublePump_',period=5000)
myCallback2 = EarlyStoping()

# Define the model and compile it
model = dde.Model(data, net)
model.compile("adam", lr=1e-4)

# Train the model and print a message when finished
losshistory, train_state = model.train(epochs=120000, callbacks=[myCallback1, myCallback2])
print('Train finish : {0} width {1} depth'.format(width, depth))

# Create the directory to save the YZ data in
if not os.path.exists(YZ_dir_path):
    os.makedirs(YZ_dir_path)

# Define a function to generate the YZ data
def plane_YZ(x=0, sample_num = 400):
    radius_x = np.sqrt(rcell**2 -x**2)
    sample_data = []

    for theta in np.linspace(0, 2*np.pi, sample_num):
        for r_pow in np.linspace(0, radius_x**2, sample_num):
            y = np.sqrt(r_pow) * np.cos(theta)
            z = np.sqrt(r_pow) * np.sin(theta)
            sample_data.append((x, y, z)) # 200*200 = 40000个采样点

    sample_data = np.asarray(sample_data, dtype=np.float32)
    return sample_data

# Generate the YZ data and save it to a file
YZ_axis = plane_YZ(0, sample_num)
y_pred_YZ = model.predict(YZ_axis)[:, 0]
y_pred_YZ = np.reshape(y_pred_YZ, (sample_num, sample_num))
YZ_df = pd.DataFrame(y_pred_YZ.T)
YZ_file_path = os.path.join(YZ_dir_path, config['SaveFile'])
YZ_df.to_csv(YZ_file_path)


In [1]:
# import deepxde as dde
# import numpy as np

# import yaml
# import pandas as pd
# import time
# import datetime
# import os
# import torch
# from scipy.special import gamma
# from scipy.special import lambertw
# import matplotlib.pyplot as plt
# from PIL import Image

# # Set default tensor type to cuda tensor
# torch.set_default_tensor_type(torch.cuda.FloatTensor)

# # Set copy flag to 0 and directory path to the YZ directory
# CopyFlag = 0
# YZ_dir_path = './result/Forward/SinglePump/'

# # Load config file
# with open('./Config/DoubleBeam.yaml') as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)
#     print(config)

# # Load sample number and YZ directory path from the config file
# sample_num = config['sample_num']
# YZ_dir_path = config['YZ_dir_path']

# # Initialize constants
# N = 1000 # bounces number
# T = 180
# D = 0.7098 #cm²/s
# I = 5*10 # kg*(cm)^2/s^3
# q = 5
# Rwall = 3.1135  #1/s
# Rsd = 15.017   #1/s
# Rrel = Rsd + q*Rwall
# k1 = 10**(-20)*10**8 # kg*cm^4/s^2
# k2 = 10**(-17)*10**4 # cm^2
# k3 = 1 # cm^-3
# sigm = 2.34601*10**(-13) # cm^2
# n = 6.0952E13 #1/cm³
# rcell = 1.5
# rpump = 0.75
# vKK = 701.27 * 100 #cm/s


Using backend: pytorch



{'ConfigDate': datetime.date(2022, 7, 15), 'CSVPath': './result/TrainResult0715.csv', 'LightIntensity': 25, 'Temperature': 160, 'NetDepth': [4], 'NetWidth': [120], 'TrainEpoch': 50000}


In [1]:
# import numpy as np

# a = np.array((0.62, 0.47, 0.31, 0.24,0.188, 0.10, 0.06))
# b = np.array((0.64, 0.54, 0.46, 0.44, 0.46, 0.54, 0.64))

# print(np.std(a))
# print(np.std(b))

0.18645259834223665
0.07772268862134539


In [2]:
 import torch

In [3]:
# def bloch_pde(x_in, y_in):
#     ''' x_in: (x,y,z) axis
#         y_in: Pz polarization
#     '''
#     x_in_copy = x_in.detach().cpu().numpy()
#     x = x_in_copy[:, 0:1]
#     y = x_in_copy[:, 1:2]
#     z = x_in_copy[:, 2:3]
    
# #     sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
# #     x_arr = x.eval(session=sess)
#     dy_xx = dde.grad.hessian(y_in, x_in, i=0, j=0)
#     dy_yy = dde.grad.hessian(y_in, x_in, i=1, j=1)
#     dy_zz = dde.grad.hessian(y_in, x_in, i=2, j=2) # dy/dxidxj
    
#     plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
#     plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
#     #Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2)) 双向泵浦光
#     Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2)) #双向泵浦光
#     Rop0_z = np.asarray(Rop0_z)    
    
#     Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
#     Rop = torch.from_numpy(Rop).cuda()
#     #Rop = Rop0
    
#     return -D*(dy_xx+dy_yy+dy_zz) + ((Rop+Rsd)/q)*y_in - Rop/q


# def func_boundary(x_in, Pz, numpy_X):
#     x_in_copy = x_in.detach().cpu().numpy()
#     x = x_in_copy[:, 0:1]
#     y = x_in_copy[:, 1:2]
#     z = x_in_copy[:, 2:3]
    
#     dPz_x = dde.grad.jacobian(Pz, x_in, i=0, j=0) # This is right
#     dPz_y = dde.grad.jacobian(Pz, x_in, i=0, j=1)
#     dPz_z = dde.grad.jacobian(Pz, x_in, i=0, j=2)
    
#     #Rop = Rop0 * np.exp(-2*(x**2+y**2)/rpump**2)
    
#     plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
#     plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
#     Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2)) #双向泵浦光
#     Rop0_z = np.asarray(Rop0_z)      
    
#     Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    
#     #second_term = np.sqrt(1/2*np.abs(Rop*D))
#     second_term = vKK/(2*N*(2-1/N))
    
#     # tranfer back to torch.tensor
#     #second_term = torch.from_numpy(second_term).cuda()
#     Rop = torch.from_numpy(Rop).cuda()
#     x = torch.from_numpy(x).cuda()
#     y = torch.from_numpy(y).cuda()
#     z = torch.from_numpy(z).cuda()
    
#     return (D/rcell*(x*dPz_x + y*dPz_y + z*dPz_z) + Pz*second_term)

In [4]:
# from utils.Mycallbacks import *


In [5]:
# geom = dde.geometry.Sphere([0, 0, 0], rcell)
# bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary) # 函数调研下，operatorBC

# data = dde.data.PDE(geom, bloch_pde, bc, num_domain=20000, num_boundary=2000)
# ModelSaveDir = config['ModelSaveDir']
# if not os.path.exists(ModelSaveDir):
#     os.makedirs(ModelSaveDir)
# ## 核心训练程序
# depth = 4
# width = 120

# # for depth in config['NetDepth']:
# #     for width in config['NetWidth']:
# net = dde.nn.FNN([3] + [width] * depth + [1], "tanh", "Glorot normal")
# # net.apply_output_transform(
# #     lambda x, y: (1 - tf.reduce_sum(x ** 2, axis=1, keepdims=True)) * y
# # )
# myCallback1 = saveModel(ModelSaveDir,'DoublePump_',period=5000)
# myCallback2 = EarlyStoping()
# model = dde.Model(data, net)
# model.compile("adam", lr=1e-4)
# #losshistory, train_state = model.train(epochs=8000, model_save_path='Bloch_Rop_v1')
# losshistory, train_state = model.train(epochs=120000, callbacks=[myCallback1, myCallback2])
# print('Train finish : {0} width {1} depth'.format(width, depth))


# if not os.path.exists(YZ_dir_path):
#     os.makedirs(YZ_dir_path)
# #         zAxis_pred = model.predict(Z_Axis())[:, 0]
# #         zAxis_df = pd.DataFrame(zAxis_pred)
# #         zAxis_path = os.path.join(YZ_dir_path, 'zAxis_pred.csv')
# #         zAxis_df.to_csv(zAxis_path)       
# #         z_fitness = fitness_metric(zAxis_pred, ZAxis_COMSOL)

# ## 计算三维数据预测结果
# # y_pred_3D = model.predict(axis_3D_axis)[:, 0]
# # y_ref = axis_3D_Pz

# # #L2_mean = np.linalg.norm(y_pred_3D - y_ref, ord=2) # 开了跟号
# # L2_error = np.mean(np.sqrt((y_pred_3D-y_ref)**2))
# # print('Evaluation : ', L2_error)
# # ## 按照实验时间，保存实验结果
# # date_time =  datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # 2022-06-18 13:49
# # record_path = os.path.join(YZ_dir_path , 'evaluation.csv')

# # if not os.path.exists(record_path):
# #     os.mkdir('./result/')

# # EvaluateDf = pd.DataFrame({
# #     'Date': [date_time],
# #     'Epoch': [config['TrainEpoch']],
# #     'Intensity(mW)': [config['LightIntensity']],
# #     'Temperature' : [config['Temperature']],
# #     'Network depth' : [depth],
# #     'Network width' : [width],
# #     #'Z axis fitness' : [z_fitness],
# #     'L2 mean error': [L2_error]
# # })    

# # EvaluateDf.to_csv(record_path, index=False, mode='a')

# ## 保存截面数据, X=0的YZ平面，坐标轴生成



# def plane_YZ(x=0, sample_num = 400):
#     radius_x = np.sqrt(rcell**2 -x**2)
#     sample_data = []    

#     for theta in np.linspace(0, 2*np.pi, sample_num):
#         for r_pow in np.linspace(0, radius_x**2, sample_num):
#             y = np.sqrt(r_pow) * np.cos(theta)
#             z = np.sqrt(r_pow) * np.sin(theta)
#             sample_data.append((x, y, z)) # 200*200 = 40000个采样点

#     sample_data = np.asarray(sample_data, dtype=np.float32
#                             )
#     return sample_data

# YZ_axis = plane_YZ(0, sample_num)
# y_pred_YZ = model.predict(YZ_axis)[:, 0]
# y_pred_YZ = np.reshape(y_pred_YZ, (sample_num, sample_num))
# YZ_df = pd.DataFrame(y_pred_YZ.T)
# YZ_file_path = os.path.join(YZ_dir_path, config['SaveFile'])    
# YZ_df.to_csv(YZ_file_path)   

Compiling model...
'compile' took 0.000276 s

Training model...

Step      Train loss              Test loss               Test metric
0         [9.05e+00, 1.92e+05]    [9.05e+00, 1.92e+05]    []  
1000      [7.97e+00, 3.26e+00]    [7.97e+00, 3.26e+00]    []  
2000      [7.97e+00, 2.21e+00]    [7.97e+00, 2.21e+00]    []  
3000      [7.97e+00, 1.29e+00]    [7.97e+00, 1.29e+00]    []  
4000      [7.96e+00, 6.12e-01]    [7.96e+00, 6.12e-01]    []  
5000      [7.94e+00, 1.19e+00]    [7.94e+00, 1.19e+00]    []  
6000      [7.95e+00, 1.26e-01]    [7.95e+00, 1.26e-01]    []  
7000      [7.93e+00, 1.33e-01]    [7.93e+00, 1.33e-01]    []  
8000      [7.94e+00, 1.04e+00]    [7.94e+00, 1.04e+00]    []  
9000      [7.91e+00, 5.43e-02]    [7.91e+00, 5.43e-02]    []  
10000     [7.91e+00, 4.54e-01]    [7.91e+00, 4.54e-01]    []  
11000     [7.89e+00, 2.29e-02]    [7.89e+00, 2.29e-02]    []  
12000     [7.91e+00, 7.67e+00]    [7.91e+00, 7.67e+00]    []  
13000     [7.86e+00, 1.93e-02]    [7.86e+00, 1